## _*QISKit ACQUA Chemistry advanced how to*_

This notebook demonstrates how to use QISKit ACQUA Chemistry to compute the ground state energy of a Lithium Hydride (LiH) molecule using VQE and UCCSD.

This notebook shows how to experiment with the ground state energy of a molecule step by step by using low-level programming API as compared to simpliy feeding a configuraiton dict as in [acqua_chemistry_howto](acqua_chemistry_howto.ipynb) example.

Here we demonstrate
  1. how to define a molecule and get integrals from a driver (PySCF)
  2. how to construct a fermionic operator and freeze/eliminate the orbitals
  3. how to map a fermionic operator to a qubit operator
  4. how to get dynamically-loadded instance, such as algorithm, optimizer, variational_form, and initial_state
  5. how to configure above instance programmatically
  6. how to retrieve the results from the algorithm

In [1]:
# import common packages
import numpy as np
import copy

import qiskit
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit

# lib from QISKit ACQUA Chemistry
from qiskit_acqua_chemistry import FermionicOperator

# lib from QISKit ACQUA
from qiskit_acqua.operator import Operator
from qiskit_acqua import (get_algorithm_instance, get_optimizer_instance, get_variational_form_instance, get_initial_state_instance)

# lib for driver
from qiskit_acqua_chemistry.drivers import ConfigurationManager
from collections import OrderedDict

import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [2]:
# using driver to get fermionic Hamiltonian
# PySCF example
cfg_mgr = ConfigurationManager()
pyscf_cfg = OrderedDict([('atom', 'Li .0 .0 .0; H .0 .0 1.6'), ('unit', 'Angstrom'), ('charge', 0), ('spin', 0), ('basis', 'sto3g')])
section = {}
section['properties'] = pyscf_cfg
driver = cfg_mgr.get_driver_instance('PYSCF')
molecule = driver.run(section)

In [3]:
# set up the to-be-frozen and to-be-removed orbitals
# please be aware that the idx here with respective to original idx
freeze_list = [0]
remove_list = [-3, -2] # negative number denotes the reverse order

h1 = molecule._one_body_integrals
h2 = molecule._two_body_integrals
nuclear_repulsion_energy = molecule._nuclear_repulsion_energy

num_electrons = molecule._num_alpha + molecule._num_beta
num_spin_orbitals = molecule._num_orbitals * 2
print("HF energy: {}".format(molecule._hf_energy - molecule._nuclear_repulsion_energy))
print("# of electrons: {}".format(num_electrons))
print("# of spin orbitals: {}".format(num_spin_orbitals))

# prepare full idx of freeze_list and remove_list
# convert all negative idx to positive
remove_list = [x % molecule._num_orbitals for x in remove_list]
freeze_list = [x % molecule._num_orbitals for x in freeze_list]
# update the idx in remove_list of the idx after frozen, since the idx of orbitals are changed after freezing
remove_list = [x - len(freeze_list) for x in remove_list]
remove_list += [x + molecule._num_orbitals - len(freeze_list)  for x in remove_list]
freeze_list += [x + molecule._num_orbitals for x in freeze_list]

HF energy: -8.821861340282716
# of electrons: 4
# of spin orbitals: 12


In [4]:
# prepare fermionic hamiltonian with orbital freezing and eliminating, and then map to qubit hamiltonian
# and if PARITY mapping is selected, reduction qubits
energy_shift = 0.0
map_type = 'PARITY'
qubit_reduction = True if map_type == 'PARITY' else False

ferOp = FermionicOperator(h1=h1, h2=h2)
if len(freeze_list) > 0:
    ferOp, energy_shift = ferOp.fermion_mode_freezing(freeze_list)
    num_spin_orbitals -= len(freeze_list)
    num_electrons -= len(freeze_list)
if len(remove_list) > 0:
    ferOp = ferOp.fermion_mode_elimination(remove_list)
    num_spin_orbitals -= len(remove_list)

qubitOp = ferOp.mapping(map_type=map_type, threshold=0.00000001)
qubitOp = qubitOp.two_qubit_reduced_operator(num_electrons) if qubit_reduction else qubitOp
qubitOp.chop(10**-10)

In [5]:
# Using exact eigensolver to get the smallest eigenvalue
exact_eigensolver = get_algorithm_instance('ExactEigensolver')
exact_eigensolver.init_args(qubitOp, k=1)
ret = exact_eigensolver.run()
print('The computed energy is: {:.12f}'.format(ret['eigvals'][0].real))
print('The exact ground state energy is: {:.12f}'.format(ret['eigvals'][0].real + energy_shift + nuclear_repulsion_energy))

The computed energy is: -1.077062771826
The exact ground state energy is: -7.881071908676


In [6]:
# setup VQE 
# setup optimizer, use L_BFGS_B optimizer for example
max_eval = 200
# lbfgs = get_optimizer_instance('L_BFGS_B')
# lbfgs.set_options(factr=10, iprint=1, maxfun=max_eval)
# spsa = get_optimizer_instance('SPSA')
# spsa.init_args(max_trials=max_eval)
cobyla = get_optimizer_instance('COBYLA')
cobyla.set_options(maxiter=max_eval)

# setup variation form generator (generate trial circuits for VQE)
HF_state = get_initial_state_instance('HartreeFock')
HF_state.init_args(qubitOp.num_qubits, num_spin_orbitals, map_type, qubit_reduction, num_electrons)
var_form = get_variational_form_instance('UCCSD')
var_form.init_args(qubitOp.num_qubits, depth=1, num_orbitals=num_spin_orbitals, num_particles = num_electrons, 
                   active_occupied=[0], active_unoccupied=[0, 1],
                   initial_state=HF_state, qubit_mapping=map_type, 
                   two_qubit_reduction=qubit_reduction, num_time_slices=1)

init_points = var_form.preferred_init_points
vqe_algorithm = get_algorithm_instance('VQE')
vqe_algorithm.setup_quantum_backend(backend='local_statevector_simulator')
vqe_algorithm.init_args(qubitOp, 'matrix', var_form, cobyla, opt_init_point=init_points)
results = vqe_algorithm.run()
print('The computed ground state energy is: {:.12f}'.format(results['eigvals'][0]))
print('The exact ground state energy is: {:.12f}'.format(results['eigvals'][0] + energy_shift + nuclear_repulsion_energy))
print("Parameters: {}".format(results['opt_params']))

The computed ground state energy is: -1.077062762924
The exact ground state energy is: -7.881071899774
Parameters: [-0.03610072 -0.00547355 -0.03596927 -0.00549639 -0.03871587  0.0604038
  0.06042029 -0.11646837]
